# Data management for the SIM model

Initial data handling for preparing material to be used in [SpatInteModel.ipynb](SpatInteModel.ipynb).

## Initialisation

In [16]:
# Loading libraries
import os
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt

# Dealing with the data
## Paths definition 
base_dir = str(Path(os.getcwd()).parent)  # Get main directory (should be parent to this file)
data_dir = os.path.join(base_dir,
                        "data",
                        "national_data") 
results_dir = os.path.join(base_dir,
                           "output")

if os.path.basename(base_dir) != "FutureHighStreets": #base_dir.split("/")[-1] != "FutureHighStreet":
    raise Exception(f"The base directory should point to the main project directory,"
                    f"but it points to {base_dir}")
    
## Variables definition
origin_data_folder = "LSOA"
origin_data_filename = "Lower_Layer_Super_Output_Areas_(December_2011)_Population_Weighted_Centroids"  # change depending on costum edits
origin_layer_name = ""
destination_data_folder = "Retail_Centres"
destination_data_filename = "Retail_Centres_UK" # change depending on costum edits
destination_gpkg_path = os.path.join(data_dir,
                         destination_data_folder,
                         destination_data_filename) + ".gpkg"

## Loading data
# origin_table = pd.read_csv(os.path.join(data_dir,
#                                       origin_data_folder,
#                                       origin_data_filename))
# origin_csv.head() 
# destination_csv = pd.read_csv(os.path.jpin(data_dir,
#                                            destiantion_data_folder,
#                                            destination_data_filename))

destination_gpkg = gpd.read_file(destination_gpkg_path,
                    layer = destination_data_filename)

# gdf = gpd.read_file(r'C:\folder\geopackage.gpkg', layer='somelayername')

destination_gpkg.head()
# crs(origin_shp) = "+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +units=m +no_defs"
# destination_shp = readOGR(dsn = data_path,
#                            layer = destination_data_filename)
# summary(destination_shp)
# beta = 0.34 # Souce: https://eprints.whiterose.ac.uk/134554/1/Wadington%20et%20al_Accounting%20for%20Temporal%20Demand.pdf

# attractiveness = as.integer(destination_shp@data$Rank)

# students_per_lsoa = as.integer(origin_shp@data$STUD_TOT)

DriverError: destination_gpkg_path: No such file or directory